## Aim: Find the top environmental projects listed on globalgiving.org.
I did this using the global giving api (https://www.globalgiving.org/api/) to provide a way, in the future, to quickly scale up the search and parse through more results.

**Tools used:** 
* GlobalGiving API
* Pandas to get data in presentable final format
* BeautifulSoup to parse through the API's XML format
* Requests instead of cURL because I prefer to visualize process in Jupyter

In [1]:
import pandas as pd
import requests
import io
from bs4 import BeautifulSoup

api_key = XXXXX

In [133]:
# pass the necessary parameters, as instructed by https://www.globalgiving.org/api/methods/
parameters = {'api_key':api_key,'q':'*','filter':'theme:env'}

In [134]:
theme_res = requests.get('https://api.globalgiving.org/api/public/projectservice/themes',params=parameters)

In [135]:
encoding = theme_res.encoding
text = theme_res.text
print(encoding)

UTF-8


In [136]:
# function to parse through requests XML output using BeautifulSoup
def parse_source(xml, encoding='utf-8'):
    parsed = BeautifulSoup(xml, from_encoding=encoding)
    return parsed

In [139]:
parsed = parse_source(text,"xml_parser")

### Begin experimenting with pulling different subheaders of content from parsed XML

In [140]:
id_rows = parsed.find_all('id')
name_rows = parsed.find_all('name')

In [141]:
l = []
s = []

for row in range(len(id_rows)):
    new_row = id_rows[row].text
    new_name = name_rows[row].text
    l.append(new_row)
    s.append(new_name)

In [142]:
df = pd.DataFrame({"ID": l,"Name": s})

In [145]:
# checking output of query
df.head()

,ID,Name
0,animals,Animals
1,children,Children
2,climate,Climate Change
3,democ,Democracy and Governance
4,disaster,Disaster Recovery


### Now that I'm familiar with the API, I'm going to pull the top environmental projects. 
Since XML format has multiple nested layers, I'm only pulling relevant, non-nested info

In [144]:
projects_res = requests.get('https://api.globalgiving.org/api/public/services/search/projects/summary',params=parameters)

In [146]:
env_encoding = projects_res.encoding
env_text = projects_res.text

In [147]:
parsed = parse_source(env_text,"xml_parser")

In [148]:
mission_rows = parsed.find_all('mission')
title_rows = parsed.find_all('title')
summary_rows = parsed.find_all('summary')
url_rows = parsed.find_all('url')

In [149]:
# correct for error in summary rows
summary_rows = summary_rows[1:]

In [151]:
a = []
b = []
c = []
d = []

In [152]:
for row in range(len(mission_rows)):
    new_mission = mission_rows[row].text
    new_title = title_rows[row].text
    new_summary = summary_rows[row].text
    new_url = url_rows[row].text

    a.append(new_mission)
    b.append(new_title)
    c.append(new_summary)
    d.append(new_url)    

In [153]:
new_df = pd.DataFrame({"Mission": a,"Title": b, "Summary": c, "URL": d})

In [155]:
new_df.head()

,Mission,Summary,Title,URL
0,VISION: To see all children enter adulthood wi...,"Join Keep The Dream196 in changing 2,587 South...","Lifeskills for 2,587 Children in South Africa",http://www.keepthedream196.com
1,The High Atlas Foundation (HAF) is dedicated t...,"Sami's Project will plant 5,000 fruit and nut ...",Improve Rural Moroccan Schools: Sami's Project,http://www.highatlasfoundation.org
2,We aim to revolutionize global health with com...,Jake's Diapers serves babies around the world ...,#savethebabies: Help Children Rise out of Poverty,http://www.jakesdiapers.org
3,Fellow Mortals is more than a place. It is a l...,Fellow Mortals provides compassionate and prof...,Fellow Mortals--Compassion Changes Everything,http://www.fellowmortals.org
4,"To provide protection for those who, by no fau...","Mickaboo Companion Bird Rescue, based in North...",Rescue Unwanted and Abandoned Companion Birds,http://www.mickaboo.org


In [128]:
new_df.to_csv('wren_new_projects.csv')